<a href="https://colab.research.google.com/github/gnitish18/Multi-Robot_Reinforcement_Learning/blob/main/foospong_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pygame, sys, time, random, os
from pygame.locals import *
import argparse
import math
import numpy as np
import pickle

import tensorflow as tf

import os
os.environ['SDL_VIDEODRIVER']='dummy'
# from objectClasses_coupled import *
# from train_from_memories_coupled import *

In [25]:

class fRect:
    """Like PyGame's Rect class, but with floating point coordinates"""

    def __init__(self, pos, size):
        self.pos = (pos[0], pos[1])
        self.size = (size[0], size[1])
    def move(self, x, y):
        return fRect((self.pos[0]+x, self.pos[1]+y), self.size)

    def move_ip(self, x, y, move_factor = 1):
        self.pos = (self.pos[0] + x*move_factor, self.pos[1] + y*move_factor)

    def get_rect(self):
        return Rect(self.pos, self.size)

    def copy(self):
        return fRect(self.pos, self.size)

    def intersect(self, other_frect):
        # two rectangles intersect iff both x and y projections intersect
        for i in range(2):
            if self.pos[i] < other_frect.pos[i]: # projection of self begins to the left
                if other_frect.pos[i] >= self.pos[i] + self.size[i]:
                    return 0
            elif self.pos[i] > other_frect.pos[i]:
                if self.pos[i] >= other_frect.pos[i] + other_frect.size[i]:
                    return 0
        return 1 #self.size > 0 and other_frect.size > 0


class Paddle:
    def __init__(self, pos, size, speed, max_angle,  facing, timeout, id):
        self.frect = fRect((pos[0]-size[0]/2, pos[1]-size[1]/2), size)
        self.speed = speed
        self.size = size
        self.facing = facing
        self.max_angle = max_angle
        self.timeout = timeout
        self.id = id
        #self.tf_model = tf_model

    def factor_accelerate(self, factor):
        self.speed = factor*self.speed

    def move(self, i, paddles, balls, table_size, states, withTFmodel, e):
        
        closest_distance = 10000
        closest_ball = None
        for ball in balls:
            # Checks distance to each ball
            if np.linalg.norm(np.asarray(ball.get_center()) - np.asarray(self.frect.pos)) < closest_distance:
                closest_distance = np.linalg.norm(np.asarray(ball.get_center()) - np.asarray(self.frect.pos))
                closest_ball = ball
            
        
        direction = self.move_getter(withTFmodel, e, states, self.id, self.frect.copy(), closest_ball.frect.copy(), tuple(table_size))
        
        
        if direction == "up":
            self.frect.move_ip(0, -self.speed)
        elif direction == "down":
            self.frect.move_ip(0, self.speed)

#        for j in range(len(paddles)):
#            if paddles[j].facing == self.facing and i != j:
#
#                # bottom of current paddle - top of other paddle (on top of other)
#                if ((self.frect.pos[1] + self.frect.size[1]) - (paddles[j].frect.pos[1])) < 0:
#                    self.frect.move_ip(0, ((self.frect.pos[1]+self.frect.size[1]) - (paddles[j].frect.pos[1])))
#
#                # bottom of other paddle - top of current paddle (below other)
#                elif ((paddles[j].frect.pos[1] + paddles[j].frect.size[1]) - self.frect.pos[1]) < 0:
#                    self.frect.move_ip(0, -((paddles[j].frect.pos[1] + paddles[j].frect.size[1]) - self.frect.pos[1]))
#

        to_bottom = (self.frect.pos[1]+self.frect.size[1])-table_size[1]
        if to_bottom > 0:
            self.frect.move_ip(0, -to_bottom)
            
        to_top = self.frect.pos[1]
        if to_top < 0:
            self.frect.move_ip(0, -to_top)
        
        if direction == "up":
            return 1
        else:
            return 0


    def get_face_pts(self):
        return ((self.frect.pos[0] + self.frect.size[0]*self.facing, self.frect.pos[1]),
                (self.frect.pos[0] + self.frect.size[0]*self.facing, self.frect.pos[1] + self.frect.size[1]-1)
                )

    def get_angle(self, y):
        center = self.frect.pos[1]+self.size[1]/2
        rel_dist_from_c = ((y-center)/self.size[1])
        rel_dist_from_c = min(0.5, rel_dist_from_c)
        rel_dist_from_c = max(-0.5, rel_dist_from_c)
        sign = 1-2*self.facing

        return sign*rel_dist_from_c*self.max_angle*math.pi/180





class Ball:
    def __init__(self, table_size, size, paddle_bounce, wall_bounce, dust_error, init_speed_mag):
        rand_ang = (.4+.4*random.random())*math.pi*(1-2*(random.random()>.5))+.5*math.pi
        speed = (init_speed_mag*math.cos(rand_ang), init_speed_mag*math.sin(rand_ang))
        pos = (table_size[0]/2, table_size[1]/2)
        self.frect = fRect((pos[0]-size[0]/2, pos[1]-size[1]/2), size)
        self.speed = speed
        self.size = size
        self.paddle_bounce = paddle_bounce
        self.wall_bounce = wall_bounce
        self.dust_error = dust_error
        self.init_speed_mag = init_speed_mag
        self.prev_bounce = None
        self.lastPaddleIdx = -1

    def get_center(self):
        return (self.frect.pos[0] + .5*self.frect.size[0], self.frect.pos[1] + .5*self.frect.size[1])


    def get_speed_mag(self):
        return math.sqrt(self.speed[0]**2+self.speed[1]**2)

    def factor_accelerate(self, factor):
        self.speed = (factor*self.speed[0], factor*self.speed[1])



    def move(self, paddles, table_size, move_factor):
        moved = 0
        paddled = 0
        walls_Rects = [Rect((-100, -100), (table_size[0]+200, 100)),
                       Rect((-100, table_size[1]), (table_size[0]+200, 100))]

        for wall_rect in walls_Rects:
            if self.frect.get_rect().colliderect(wall_rect):
                c = 0
                
                while self.frect.get_rect().colliderect(wall_rect):
                    self.frect.move_ip(-.1*self.speed[0], -.1*self.speed[1], move_factor)
                    c += 1 # this basically tells us how far the ball has traveled into the wall
                r1 = 1+2*(random.random()-.5)*self.dust_error
                r2 = 1+2*(random.random()-.5)*self.dust_error

                self.speed = (self.wall_bounce*self.speed[0]*r1, -self.wall_bounce*self.speed[1]*r2)
                
                while c > 0 or self.frect.get_rect().colliderect(wall_rect):
                    self.frect.move_ip(.1*self.speed[0], .1*self.speed[1], move_factor)
                    c -= 1 # move by roughly the same amount as the ball had traveled into the wall
                moved = 1
                

        for paddle in paddles:
            if self.frect.intersect(paddle.frect):
                if (paddle.facing == 1 and self.get_center()[0] < paddle.frect.pos[0] + paddle.frect.size[0]/2) or \
                (paddle.facing == 0 and self.get_center()[0] > paddle.frect.pos[0] + paddle.frect.size[0]/2):
                    continue
                
                c = 0
                
                while self.frect.intersect(paddle.frect) and not self.frect.get_rect().colliderect(walls_Rects[0]) and not self.frect.get_rect().colliderect(walls_Rects[1]):
                    self.frect.move_ip(-.1*self.speed[0], -.1*self.speed[1], move_factor)
                    
                    c += 1
                    
                theta = paddle.get_angle(self.frect.pos[1]+.5*self.frect.size[1])
                

                v = self.speed

                v = [math.cos(theta)*v[0]-math.sin(theta)*v[1],
                             math.sin(theta)*v[0]+math.cos(theta)*v[1]]

                v[0] = -v[0]

                v = [math.cos(-theta)*v[0]-math.sin(-theta)*v[1],
                              math.cos(-theta)*v[1]+math.sin(-theta)*v[0]]


                # Bona fide hack: enforce a lower bound on horizontal speed and disallow back reflection
#                if  v[0]*(2*paddle.facing-1) < 1: # ball is not traveling (a) away from paddle (b) at a sufficient speed
#                    v[1] = (v[1]/abs(v[1]))*math.sqrt(v[0]**2 + v[1]**2 - 1) # transform y velocity so as to maintain the speed
#                    v[0] = (2*paddle.facing-1) # note that minimal horiz speed will be lower than we're used to, where it was 0.95 prior to the  increase by 1.2

                #a bit hacky, prevent multiple bounces from accelerating
                #the ball too much
                if not paddle is self.prev_bounce:
                    self.speed = (v[0]*self.paddle_bounce, v[1]*self.paddle_bounce)
                else:
                    self.speed = (v[0], v[1])
                self.prev_bounce = paddle
                paddled = 1

                while c > 0 or self.frect.intersect(paddle.frect):
                
                    self.frect.move_ip(.1*self.speed[0], .1*self.speed[1], move_factor)
                    
                    c -= 1
                
                moved = 1
                

        if not moved:
            self.frect.move_ip(self.speed[0], self.speed[1], move_factor)
        
        return paddled


def directions_from_input(paddle_rect, ball_rect, table_size):
    keys = pygame.key.get_pressed()

    if keys[pygame.K_UP]:
        return "up"
    elif keys[pygame.K_DOWN]:
        return "down"
    else:
        return None




def timeout(func, args=(), kwargs={}, timeout_duration=1, default=None):
    '''From:
    http://code.activestate.com/recipes/473878-timeout-function-using-threading/'''
    import threading
    class InterruptableThread(threading.Thread):
        def __init__(self):
            threading.Thread.__init__(self)
            self.result = None

        def run(self):
            try:
                self.result = func(*args, **kwargs)
            except:
                self.result = default

    it = InterruptableThread()
    it.start()
    it.join(timeout_duration)
    if it.isAlive():
        print("TIMEOUT")
        return default
    else:
        return it.result

def render(screen, paddles, balls, score, table_size):
    screen.fill(black)

    for paddle in paddles:
        pygame.draw.rect(screen, white, paddle.frect.get_rect())

    for ball in balls:
        pygame.draw.circle(screen, white, (int(ball.get_center()[0]), int(ball.get_center()[1])),  int(ball.frect.size[0]/2), 0)


    pygame.draw.line(screen, white, [screen.get_width()/2, 0], [screen.get_width()/2, screen.get_height()])

    score_font = pygame.font.Font(None, 32)
    screen.blit(score_font.render(str(score[0]), True, white), [int(0.4*table_size[0])-8, 0])
    screen.blit(score_font.render(str(score[1]), True, white), [int(0.6*table_size[0])-8, 0])

    pygame.display.flip()



def check_point(score, balls, table_size):
    for i in range(len(balls)):
        ball = balls[i]
        lastPaddleIdxs = []
        if ball.frect.pos[0]+ball.size[0]/2 < 0:
            score[1] += 1
            #tracks which paddle hit the ball last, so that we can attribute the reward to the the right timestep
            if ball.prev_bounce is not None and ball.prev_bounce.facing == 0:
                lastPaddleIdxs.append(ball.lastPaddleIdx)
                
            balls[i] = Ball(table_size, ball.size, ball.paddle_bounce, ball.wall_bounce, ball.dust_error, ball.init_speed_mag)
            
        elif ball.frect.pos[0]+ball.size[0]/2 >= table_size[0]:
            balls[i] = Ball(table_size, ball.size, ball.paddle_bounce, ball.wall_bounce, ball.dust_error, ball.init_speed_mag)
            score[0] += 1
            #return (ball, score)

    return (balls, score, lastPaddleIdxs)



In [26]:

class foosPong_model(tf.keras.Model):
    def __init__(self):
        super(foosPong_model, self).__init__()
        ###############################################
        self.drop = tf.keras.layers.Dropout(0.20)
        self.gauss = tf.keras.layers.GaussianNoise(stddev=0.2)
        #self.n1 = tf.keras.layers.BatchNormalization()
        #self.n2 = tf.keras.layers.BatchNormalization()
        
        self.d1 = tf.keras.layers.Dense(48, activation='relu')
        self.d2 = tf.keras.layers.Dense(48*4, activation='relu')
        self.d3 = tf.keras.layers.Dense(48*8, activation='relu')
        self.d4 = tf.keras.layers.Dense(48*4, activation='relu')
        self.d5 = tf.keras.layers.Dense(48, activation='relu')
        
        # size 4, so that each teammate has action space of (up, down)
        # output here is Q value for each possible action for each teammate, which gets added together in loss function for total max q-value
        self.d6 = tf.keras.layers.Dense(4)
        
        ###############################################
        
    def call(self, x):
        x = self.gauss(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.drop(x)
        x = self.d3(x)
        x = self.drop(x)
        x = self.d4(x)
        x = self.drop(x)
        x = self.d5(x)
        return self.d6(x)

#def pong_ai(tensor):
#
#    if paddle_frect.pos[1] + paddle_frect.size[1]/2 < ball_frect.pos[1] + ball_frect.size[1]/2:
#        return 1
#    else:
#        return 0

def loss(curr_output, action, reward, target_output):
    gamma = 0.95

    #curr_action = tf.math.argmax(curr_output, 1)
#    curr_output = tf.round(curr_output)
#    target_output = tf.round(target_output)
    
    Q1 = tf.gather(curr_output[:, 0:2], tf.math.argmax(curr_output[:, 0:2], 1), axis=1) + tf.gather(curr_output[:, 2:4], tf.math.argmax(curr_output[:, 2:4], 1), axis=1)
    
    Q2 = tf.gather(target_output[:, 0:2], tf.math.argmax(target_output[:, 0:2], 1), axis=1) + tf.gather(target_output[:, 2:4], tf.math.argmax(target_output[:, 2:4], 1), axis=1)
    
    y = gamma*Q2 + reward[:,0]
    
    loss = tf.keras.losses.MSE(y, Q1)
    return loss
    
def train_nn(memories, curr_model, prev_model):
#################################################
### Tune these parameters for better training
    lr = 0.00000025
    #lr = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=100, decay_rate=0.5)
    epochs = 100
    batch_size = 50
  #################################################
#    lr = 0.00000025
#    epochs = 100 # best so far!
#    batch_size = 50
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    
    
    @tf.function
    def train(train_data):
        for tensor in train_data:
            train_step(tensor)


    @tf.function
    def train_step(tensor):
        state = tensor[:, :24]
        action = tensor[:, 24:26]
        reward = tensor[:, 26:28]
        next_state = tensor[:, 28:]
        
        with tf.GradientTape() as tape:
            current_loss = loss(curr_model(state), action, reward, prev_model(next_state))

        grad = tape.gradient(current_loss, curr_model.trainable_variables)
        optimizer.apply_gradients(zip(grad, curr_model.trainable_variables))
        train_loss(current_loss)

    train_data = []
#    for i in range(memories[0].shape[0]):
#        state = memories[0][i,:]
#        action = memories[1][i,:]
#        reward = memories[2][i,:]
#        next_state = memories[3][i,:]
#        train_data.append(np.concatenate((state, action, reward, next_state)))

    data_size = 10000
    #idx = int(np.floor(np.random.random()*(memories[0].shape[0] - data_size)))
    for i in range(data_size):
        idx = int(np.floor(np.random.random()*(memories[0].shape[0])))
            #if idx + i < memories[0].shape[0]:
        state = memories[0][idx,:]
        action = memories[1][idx,:]
        reward = memories[2][idx,:]
        next_state = memories[3][idx,:]
        train_data.append(np.concatenate((state, action, reward, next_state)))

    # could shuffle here. I'm unclear on randomizing each step or maintaining order
    train_data_tf = tf.data.Dataset.from_tensor_slices(train_data).shuffle(50000).batch(batch_size)
    #train_data_tf = tf.data.Dataset.from_tensor_slices(train_data).batch(batch_size)
    
    for epoch in range(epochs):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train(train_data_tf)
        #print("works")
        template = '\nEpoch {}, Loss: {}\n'
        print(template.format(epoch + 1, train_loss.result()))
        
        #updates target network
#        if epoch % 50 == 0:
#            prev_model = curr_model
    curr_model.save_weights('./trained_weights/foosPong_model_integrated')
    return curr_model
    
#    curr_model.summary()
#    #Saves model
#    curr_model.save_weights('./trained_weights/foosPong_model_v0')


In [27]:
white = [255, 255, 255]
black = [0, 0, 0]
clock = pygame.time.Clock()

class foosPong_model(tf.keras.Model):
    def __init__(self):
        super(foosPong_model, self).__init__()
        ###############################################
        self.drop = tf.keras.layers.Dropout(0.20)
        self.gauss = tf.keras.layers.GaussianNoise(stddev=0.2)
        #self.n1 = tf.keras.layers.BatchNormalization()
        #self.n2 = tf.keras.layers.BatchNormalization()
        
        self.d1 = tf.keras.layers.Dense(48, activation='relu')
        self.d2 = tf.keras.layers.Dense(48*4, activation='relu')
        self.d3 = tf.keras.layers.Dense(48*8, activation='relu')
        self.d4 = tf.keras.layers.Dense(48*4, activation='relu')
        self.d5 = tf.keras.layers.Dense(48, activation='relu')
        
        # size 4, so that each teammate has action space of (up, down)
        # output here is Q value for each possible action for each teammate, which gets added together in loss function for total max q-value
        self.d6 = tf.keras.layers.Dense(4)
        
        ###############################################
        
    def call(self, x):
        x = self.gauss(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.drop(x)
        x = self.d3(x)
        x = self.drop(x)
        x = self.d4(x)
        x = self.drop(x)
        x = self.d5(x)
        return self.d6(x)


In [28]:

def game_loop(screen, paddles, balls, table_size, clock_rate, turn_wait_rate, score_to_win, display, e, yesRender=True, withTFmodel=False):
    score = [0, 0]
    
    
    
    states = [] #state of all paddles and all balls, positions and velocities
    actions = [] #actions that each paddle takes
    rewards = [] #sum of rewards after each movement
    next_states = []

    idx = 0
    while max(score) < score_to_win:
        old_score = score[:]
        #print(idx)
        
        #balls, score = check_point(score, balls, table_size)
        
        ########### update memories with current states of paddles and balls ############################################################
        
        curr_states = []
        for paddle in paddles:
            curr_states.append(paddle.frect.pos[0])
            curr_states.append(paddle.frect.pos[1])
        for ball in balls:
            curr_states.append(ball.get_center()[0])
            curr_states.append(ball.get_center()[1])
            curr_states.append(ball.speed[0])
            curr_states.append(ball.speed[1])
        
       
        # Take actions...and add to memory actions
        curr_actions = []
        for i in range(len(paddles)):
            if paddles[i].facing == 0:
                action = paddles[i].move(i, paddles, balls, table_size, curr_states, withTFmodel, e)
                curr_actions.append(action)
            else:
                action = paddles[i].move(i, paddles, balls, table_size, curr_states, False, e)
        
        
        
        for ball in balls:
            paddled = 0
            inv_move_factor = int((ball.speed[0]**2+ball.speed[1]**2)**.5)
            if inv_move_factor > 0:
                for i in range(inv_move_factor):
                    paddled = ball.move(paddles, table_size, 1./inv_move_factor)
            else:
                paddled = ball.move(paddles, table_size, 1)
                
            if paddled == 1:
                ball.lastPaddleIdx = idx
            
       
        new_states = []
        for paddle in paddles:
            new_states.append(paddle.frect.pos[0])
            new_states.append(paddle.frect.pos[1])
        for ball in balls:
            new_states.append(ball.get_center()[0])
            new_states.append(ball.get_center()[1])
            new_states.append(ball.speed[0])
            new_states.append(ball.speed[1])
        
        
        # Check if a ball scored and add rewards accordingly, so rewards[i] should correspond to actions taken at actions[i]
        balls, score, lastPaddleIdxs = check_point(score, balls, table_size)
        
        curr_rewards = []
        if score != old_score:
            if score[0] != old_score[0]:
                #-1 for each point opponent scores
                curr_rewards.append(-50)
                curr_rewards.append(-50)
            else:
                #+1 each time our team scores
                curr_rewards.append(0)
                curr_rewards.append(0)
                for i in lastPaddleIdxs:
                    # adds reward back to the time step that a paddle on our team hit the ball
                    if i != -1:
                        #print(i)
                        #print(idx)
                        rewards[i][0] = rewards[i][0] + 100
                        rewards[i][1] = rewards[i][1] + 100
        else:
            # Reward 0 if nothing happens?
            curr_rewards.append(0)
            curr_rewards.append(0)
            
        
        
        if (np.random.random() < 1.0) or score != old_score:
            states.append(curr_states)
            actions.append(curr_actions)
            next_states.append(new_states)
            rewards.append(curr_rewards)
            idx = idx + 1
        


################       SCREEN RENDER       ########################

        if yesRender:
            render(screen, paddles, balls, score, table_size)

##########################################################################

    for i in range(len(balls)):
            balls[i] = Ball(table_size, ball.size, ball.paddle_bounce, ball.wall_bounce, ball.dust_error, ball.init_speed_mag)
    
    print(score)
    print("idx", idx)
    print("states: ", len(states), "actions: ", len(actions), "rewards: ", len(rewards), "next_states: ", len(next_states))
    return states, actions, rewards, next_states


In [29]:

def init_game(args):
    table_size = (800, 600)
    paddle_size = (5, 70)
    ball_size = (15, 15)
    paddle_speed = 5 #1
    max_angle = 45

    paddle_bounce = 1.5 #1.2
    wall_bounce = 1.00
    dust_error = 0.00
    init_speed_mag = 2
    timeout = 0.0003
    clock_rate = 200 #80
    turn_wait_rate = 3
    score_to_win = 10


    screen = pygame.display.set_mode(table_size)
    pygame.display.set_caption('PongAIvAI')


    paddles = [Paddle((30, table_size[1]/4), paddle_size, paddle_speed, max_angle,  1, timeout, 0), \
               Paddle((300, table_size[1] - table_size[1]/4), paddle_size, paddle_speed, max_angle,  1, timeout, 1), \
               Paddle((table_size[0] - 30, table_size[1]/4), paddle_size, paddle_speed, max_angle,  0, timeout, 0), \
               Paddle((table_size[0] - 300, table_size[1] - table_size[1]/4), paddle_size, paddle_speed, max_angle, 0, timeout, 1)]
               
    #ball = Ball(table_size, ball_size, paddle_bounce, wall_bounce, dust_error, init_speed_mag)
    balls = [Ball(table_size, ball_size, paddle_bounce, wall_bounce, dust_error, init_speed_mag), Ball(table_size, ball_size, paddle_bounce, wall_bounce, dust_error, init_speed_mag), Ball(table_size, ball_size, paddle_bounce, wall_bounce, dust_error, init_speed_mag), Ball(table_size, ball_size, paddle_bounce, wall_bounce, dust_error, init_speed_mag)]
    
    
    
    def pong_ai(paddle_frect, ball_frect, table_size):
        if paddle_frect.pos[1] + paddle_frect.size[1]/2 < ball_frect.pos[1] + ball_frect.size[1]/2:
           return "down"
        else:
           return  "up"
    
    def foosPong_ai(states, id):
        output = foosPong(np.asarray(states, dtype='float32').reshape((1,24)))
        team_Q_values = tf.reshape(output, [2,2])
        action_idx = tf.math.argmax(team_Q_values[id,:]).numpy()
        
        if action_idx == 0:
            return "down"
        else:
            return "up"
        
    def move_getter(withTFmodel, e, states, id, paddle_frect, ball_frect, table_size):
        if withTFmodel:
            if np.random.random() < e:
                return pong_ai(paddle_frect, ball_frect, table_size)
            else:
                return foosPong_ai(states, id)
        else:
            return pong_ai(paddle_frect, ball_frect, table_size)
    
    
    # Set move getter functions
    paddles[0].move_getter = move_getter
    paddles[1].move_getter = move_getter
    paddles[2].move_getter = move_getter
    paddles[3].move_getter = move_getter
        
    foosPong = foosPong_model()
    # eps = float(args.eps)
    eps = 1.0
    yesRender = True
    withTFmodel = False
    if args == 'false': yesRender = False
    #if args.withTFmodel == 'true': withTFmodel = True
        #foosPong.load_weights('./trained_weights/foosPong_model_v0')  
    
    episodes = 1000
    memory_states = []
    memory_actions = []
    memory_rewards = []
    memory_next_states = []
    decay = 0.005
    for ep in range(episodes):
        print(f"\nEpisode: {ep}")
        ep_states, ep_actions, ep_rewards, ep_next_states = game_loop(screen, paddles, balls, table_size, clock_rate, turn_wait_rate, score_to_win, 1, eps-decay*ep, yesRender=yesRender, withTFmodel=withTFmodel)
        
        memory_states = memory_states + ep_states
        memory_actions = memory_actions + ep_actions
        memory_rewards = memory_rewards + ep_rewards
        memory_next_states = memory_next_states + ep_next_states
        print("memory_states: ", len(memory_states), "memory_actions: ", len(memory_actions), "memory_rewards: ", len(memory_rewards), "memory_next_states: ", len(memory_next_states), "\n")
        
        
        # after so many steps, take a pause
        # foosPong_model = train_nn(memories, foosPong_model)
        if len(memory_states) > 50000:
            if ep % 25 == 0:
                memories = [np.asarray(memory_states, dtype='float32'), np.asarray(memory_actions, dtype='float32'), np.asarray(memory_rewards, dtype='float32'), np.asarray(memory_next_states, dtype='float32')]
                
                foosPong = train_nn(memories, foosPong, foosPong)
            print("before", len(memory_states))
            del memory_states[0:len(ep_states)]
            del memory_actions[0:len(ep_actions)]
            del memory_rewards[0:len(ep_rewards)]
            del memory_next_states[0:len(ep_next_states)]
            print("after", len(memory_states))
            
    #    with open("memory_states.txt", "wb") as fp:
    #        pickle.dump(memory_states, fp)
    #    print("States dumped...")
    #
    #    with open("memory_actions.txt", "wb") as fp:
    #        pickle.dump(memory_actions, fp)
    #    print("Actions dumped...")
    #
    #    with open("memory_rewards.txt", "wb") as fp:
    #        pickle.dump(memory_rewards, fp)
    #    print("Rewards dumped...")
    #
    #    with open("memory_next_states.txt", "wb") as fp:
    #        pickle.dump(memory_next_states, fp)
    #    print("Next_states dumped...")
    #
    #    print(np.asarray(memory_states).shape)
    
    pygame.quit()


In [ ]:
if __name__ == '__main__':
    #parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    #parser.add_argument('--eps')
    #parser.add_argument('--yesRender')
    #parser.add_argument('--withTFmodel')

    args = 'false' 
    #parser.parse_args()
    
    pygame.init()
    init_game(args)